In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Dataset for train
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')

#Dataset for test
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')

In [ ]:
pd.set_option("display.max_columns",None)
train.head()

In [ ]:
import seaborn as sns
corr = train.corr()
#mask = np.triu(np.ones_like(corr, dtype=bool))
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(18, 18))
sns.heatmap(corr)

In [ ]:
import seaborn as sns 
sns.distplot(train["target"])

In [ ]:
train.isna().sum().reset_index()

In [ ]:
train = pd.get_dummies(train, drop_first = True)
train.head()

In [ ]:
y = train["target"]
x = train.drop(["target" , "cat6_G"], axis =1)

In [ ]:
x.head()

In [ ]:
x.shape

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x,y)

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=x.columns)

In [ ]:
feat_importances.nlargest(35).plot(kind='barh')

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [ ]:
import xgboost as xgb
regressor=xgb.XGBRegressor(n_estimators = 1100, learning_rate=0.05 , max_depth=30 ,subsample =0.8 , min_child_weight =3 )
regressor.fit(X_train,y_train)

In [ ]:
print("Coefficient of determination R^2 <-- on train set: {}".format(regressor.score(X_train, y_train)))

In [ ]:
'''from sklearn.model_selection import cross_val_score
score=cross_val_score(regressor,X,y,cv=5)
'''

In [ ]:
xg_pred=regressor.predict(X_test)

In [ ]:
sns.distplot(y_test-xg_pred)

In [ ]:
print("Coefficient of determination R^2 <-- on test set: {}".format(regressor.score(X_test, xg_pred)))

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, xg_pred))
print('MSE:', metrics.mean_squared_error(y_test, xg_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, xg_pred)))

SUBMISSIONS

In [ ]:
test = pd.get_dummies(test , drop_first = True)

In [ ]:
arr1 = X_train.columns
arr2 = test.columns
for i in arr1:
    if i not in arr2:
        print(i)

In [ ]:
len(test.columns)

In [ ]:
len(X_train.columns)

In [ ]:
target = regressor.predict(test)

In [ ]:
submissions = pd.read_csv("../input/tabular-playground-series-feb-2021/sample_submission.csv")
submissions.head()


In [ ]:
submissions.drop("target" , axis =1,inplace =True)

In [ ]:
submissions["target"] = target

In [ ]:
submissions.to_csv('sub.csv', index=False)